# 三种建模方式

## 一、Sequential model

### 第一种Sequential model

In [1]:
from tensorflow.keras import layers
import tensorflow as tf

model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))  # 第一层
model.add(layers.Dense(64, activation='relu'))  # 第二层
model.add(layers.Dense(10))  # 第三层

### 第二种Sequential model

In [2]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),  # 第一层
    layers.Dense(64, activation='relu'),  # 第二层
    layers.Dense(10)  # 第三层
])

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 44.2523 - accuracy: 0.1060
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 186.8199 - accuracy: 0.0930
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 1079.1598 - accuracy: 0.0840
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 3199.9565 - accuracy: 0.0920
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 5311.5405 - accuracy: 0.0940
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 8510.5234 - accuracy: 0.0920
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 10440.2783 - accuracy: 0.0830
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 18815.2812 - accuracy: 0.0910
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 18890.6055 - accuracy: 0.0820
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 27916.4941 - accur

tf.keras.Model.fit (用于模型训练) 

- epochs：训练分为几个时期。每一个epoch是对整个输入数据的一次迭代（此操作以较小的批次完成）。
- batch_size：当传递NumPy数据时，模型将数据切成较小的批次，并在训练期间对这些批次进行迭代。该整数指定每个批次的大小。请注意，如果不能将样本总数除以批次大小，则最后一批可能会更小。
- validation_data：在模型训练时，监控在某些验证数据上监视其性能。传递此参数（输入和标签的元组）可以使模型在每个时期结束时以推断模式显示所传递数据的损失和度量。

##  二、Functional model

复杂模型怎么办？
* 多输入模型
* 多输出模型
* 具有共享图层（同一图层被调用多次）的模型，
* 具有非顺序数据流的模型（例如，残余连接）。
* 训练模型和tf.keras.Sequential 一样

函数式模型是一种创建模型的方法，该模型比tf.keras.Sequential更灵活。函数式模型可以处理具有非线性拓扑的模型，具有共享层的模型以及具有多个输入或输出的模型等等

深度学习模型通常是层的有向无环图（DAG）的主要思想。因此，函数式模型是一种构建层图的方法。



In [5]:
inputs = tf.keras.Input(shape=(32,))  
# inputs = tf.keras.Input(shape=(32,))  
x = layers.Dense(64, activation='relu')(inputs) # 第一层
x = layers.Dense(64, activation='relu')(x)  # 第二层
predictions = layers.Dense(10)(x)  # 第三层

In [6]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 12.8718 - accuracy: 0.1110
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 21.0259 - accuracy: 0.1090
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 37.7969 - accuracy: 0.0980
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 60.2523 - accuracy: 0.1180
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 85.9100 - accuracy: 0.1230


In [7]:
inputs1 = tf.keras.Input(shape=(32,))  # 输入1
inputs2 = tf.keras.Input(shape=(32,))  # 输入2
x1 = layers.Dense(64, activation='relu')(inputs1)  # 第一层
x2 = layers.Dense(64, activation='relu')(inputs2)  # 第一层
x = tf.concat([x1,x2], axis=-1)
x = layers.Dense(64, activation='relu')(x) #第二层
predictions = layers.Dense(10)(x) #第三层

In [8]:
model = tf.keras.Model(inputs=[inputs1,inputs2], outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np
data1 = np.random.random((1000, 32))
data2 = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
model.fit((data1,data2), labels, batch_size=32, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 15.2289 - accuracy: 0.0940
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 40.7740 - accuracy: 0.1170
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 85.2273 - accuracy: 0.1070
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 136.7799 - accuracy: 0.1020
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 194.8190 - accuracy: 0.1000


## 模型子类化 (重要！！！)

通过子类化 `tf.keras.Model` 和定义自己的前向传播模型来构建完全可定制的模型。

和 `eager execution` 模式相辅相成。


In [9]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = layers.Dense(32, activation='relu') #
        self.dense_2 = layers.Dense(num_classes)

    def call(self, inputs):
        # 定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return x

In [10]:
model = MyModel(num_classes=10)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 12.4246 - accuracy: 0.0970
Epoch 2/5
32/32 [==============================] - 0s 997us/step - loss: 14.2109 - accuracy: 0.1000
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 16.3846 - accuracy: 0.1030
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 18.1348 - accuracy: 0.1030
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 19.8920 - accuracy: 0.1000
